# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118681e+02     1.565970e+00
 * time: 0.7925469875335693
     1     1.082928e+01     1.008279e+00
 * time: 3.469465970993042
     2    -1.206560e+01     1.003450e+00
 * time: 4.1813130378723145
     3    -3.417853e+01     7.904618e-01
 * time: 5.235162973403931
     4    -4.761513e+01     5.819765e-01
 * time: 6.276344060897827
     5    -5.691342e+01     2.002339e-01
 * time: 7.26018500328064
     6    -5.983628e+01     1.355862e-01
 * time: 7.932932138442993
     7    -6.094826e+01     5.086208e-02
 * time: 8.62177300453186
     8    -6.136276e+01     6.426620e-02
 * time: 9.29264211654663
     9    -6.164731e+01     3.923239e-02
 * time: 9.9654541015625
    10    -6.185857e+01     2.604440e-02
 * time: 10.66375994682312
    11    -6.201217e+01     2.217478e-02
 * time: 11.383579969406128
    12    -6.207518e+01     1.892492e-02
 * time: 12.080635070800781
    13    -6.213716e+01     1.439288e-02
 * time: 12.808558940887451
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557674
    AtomicNonlocal      14.8522636
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666460322
